In [1]:
# import libaries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import time
import matplotlib.pyplot as plt
from scipy.stats import poisson

### Step 1: Get EPL Team Overall Stats from web
Get EPL teams' stats using database in www.pesmaster.com.
The result dictionary will have 6 seasons' stats from 13/14 ~ 18/19 EPL.

In [2]:
# list of the teams (18/19)
# data from three different sources use different names for teams. 
# manually making a dictionary to rename teams
team_name = {
            'Arsenal FC': 'Arsenal',
            'Burnley FC': 'Burnley',
            'Liverpool FC': 'Liverpool',
            'Everton FC': 'Everton',
            'Fulham FC': 'Fulham',
            'Southampton FC': 'Southampton',
            'Chelsea FC': 'Chelsea',
            'West Bromwich Albion': 'West Bromwich',
            'Manchester United': 'Manchester Utd',
            'Man United': 'Manchester Utd',
            'Man City': 'Manchester City', 
            'Newcastle': 'Newcastle Utd',
            'Newcastle United': 'Newcastle Utd',
            'West Ham United': 'West Ham Utd', 
            'Tottenham Hotspur': 'Tottenham',
            'Queens Park Rangers': 'QP Rangers',
            'Watford FC': 'Watford',
            'AFC Bournemouth': 'Bournemouth',
            'Sunderland AFC': 'Sunderland',
            'Middlesbrough FC': 'Middlesbrough',
            'Brighton & Hove Albion': 'Brighton & Hove',
            'Huddersfield Town': 'Huddersfield',
            'QPR': 'QP Rangers',
            'Cardiff': 'Cardiff City',
            'Hull': 'Hull City',
            'Norwich': 'Norwich City',
            'Stoke': 'Stoke City',
            'Swansea': 'Swansea City',
            'West Brom': 'West Bromwich',
            'West Ham': 'West Ham Utd',
            'Reading FC': 'Reading',
            'Leicester': 'Leicester City'
            }

In [3]:
teams_seasons = []
for year in range(2014, 2020):
    url = 'https://www.pesmaster.com/english-league/pes-' + str(year) + '/league/9/'
    res = requests.get(url, headers={'User-agent': 'slsl'})
    
    if res.status_code != 200:
        print('Status not 200', res.status_code)
        break

    soup = BeautifulSoup(res.content, 'lxml')
    table = soup.find("table", {"id" : "search-result-table"})
    tr = table.find_all('tr')
    
    teams = []

    for i in range(1,(len(tr))):
            result = {}

            td = tr[i].find_all('td')
            
            if td:
                result['Team'] = td[0].text
                result['Ovr'] = td[1].text
                result['Def'] = td[2].text
                result['Mid'] = td[3].text
                result['Fwd'] = td[4].text
                result['Phy'] = td[5].text
                result['Spd'] = (td[6].text)

            if len(result) == 7:
                teams.append(result)
                
    time.sleep(3)
    
    teams_df = pd.DataFrame(teams, columns=(['Team', 'Ovr', 'Def', 'Mid', 'Fwd', 'Phy', 'Spd']))
    teams_df = teams_df.set_index('Team')
    teams_df.rename(index=team_name, inplace=True)

    # put all seasons into a list
    teams_seasons.append(teams_df)
    
# make a list as a dictionary that has keys = season (2013, 2014, ..., 2017)
teams_dict = {str(2014 + i) : teams_seasons[i] for i in range(0,6)}

In [4]:
teams_dict['2019'].head()

,Ovr,Def,Mid,Fwd,Phy,Spd
Team,,,,,,
Manchester City,84,83,86,85,76,80
Liverpool,83,82,82,85,75,79
Chelsea,83,82,83,85,77,78
Manchester Utd,83,81,82,85,78,78
Arsenal,82,81,82,84,76,77


In [5]:
for season in range(2014, 2020):
    teams_dict[str(season)].to_csv('./Data/PES' + str(season-2000) + '.csv')

In [6]:
# list of the teams (18/19)
pes_team_dict = {
            'ARSENAL': 'Arsenal',
            'LIVERPOOL': 'Liverpool',
            'SOUTH WALES': 'Cardiff City',
            'MAN BLUE': 'Manchester City',
            'EAST DORSETSHIRE': 'Bournemouth',
            'WEST LONDON WHITE': 'Fulham',
            'EAST MIDLANDS': 'Leicester City',
            'SOUTH NORWOOD': 'Crystal Palace',
            'TYNESIDE': 'Newcastle Utd',
            'NORTH EAST LONDON': 'Tottenham',
            'EAST LONDON': 'West Ham Utd',
            'HAMPSHIRE RED': 'Southampton',
            'LANCASHIRE CLARET': 'Burnley',
            'HERTFORDSHIRE': 'Watford',
            'MAN RED': 'Manchester Utd',
            'EAST SUSSEX': 'Brighton & Hove',
            'LONDON FC': 'Chelsea',
            'MERSEYSIDE BLUE': 'Everton',
            'WM GOLD':'Wolverhampton',
            'WEST YORKSHIRE TOWN': 'Huddersfield',
            'WEST GLAMORGAN CITY': 'Swansea City'}

In [7]:
players = []

for page in range(1,18):
    url = 'http://pesdb.net/pes2019/?league=1&sort=club_team&order=a&page=' + str(page)
    res = requests.get(url, headers={'User-agent': 'slsl'})
    
    if res.status_code != 200:
        print('Status not 200', res.status_code)
        break

    soup = BeautifulSoup(res.content, 'lxml')
    table = soup.find("table", {"class" : "players"})
    tr = table.find_all('tr')

    for i in range(1,(len(tr))):
            result = {}

            td = tr[i].find_all('td')
            
            if td:
                result['Name'] = td[1].text
                result['Team'] = td[2].text
                result['Rating'] = (td[9].text)

            if len(result) == 3:
                players.append(result)
                
    time.sleep(3)
    
players_df = pd.DataFrame(players, columns=(['Name', 'Team', 'Rating']))
final_player = players_df.set_index('Team')
final_player.rename(index=pes_team_dict, inplace=True)


In [8]:
final_player.head()

,Name,Rating
Team,,
Arsenal,P. AUBAMEYANG,88
Arsenal,M. ÖZIL,87
Arsenal,H. MKHITARYAN,85
Arsenal,A. LACAZETTE,85
Arsenal,B. LENO,85


### Step 2: Get EPL Tables of Last 5 Seasons (2013~2018)

In [9]:
season_tables = []

for year in range(2013, 2019):
    # load data from soccerstats.com by years (2013 will mean 12/13)
    table_url = 'https://www.soccerstats.com/widetable.asp?league=england_' + str(year)
    res = requests.get(table_url, headers={'User-agent': 'slsl'})

    if res.status_code != 200:
        print('Status not 200', res.status_code)
        break

    soup = BeautifulSoup(res.content, 'lxml')
    
    # get a table
    table = soup.find('table', {'id': 'btable'})
    tr = table.find_all('tr', {'class': 'trow8'})

    final_table = []

    for i in range(0,(len(tr))):
        result = {}

        td = tr[i].find_all('td')

        if td:
            result['Team'] = td[1].text.strip()     # team name
            result['P'] = int(td[2].text)           # number of games
            result['W'] = int(td[3].text)           # wins
            result['D'] = int(td[4].text)           # draws
            result['L'] = int(td[5].text)           # loses
            result['HW'] = int(td[13].text)         # home wins
            result['HGF'] = int(td[16].text)        # home goals scored
            result['HGA'] = int(td[17].text)        # home goals conceded
            result['AGF'] = int(td[27].text)        # away goals scored
            result['AGA'] = int(td[28].text)        # away goals conceded
            result['AW'] = int(td[24].text)         # away wins
            result['GF'] = int(td[6].text)          # goals for
            result['GA'] = int(td[7].text)          # goals allowed
            result['PTS'] = int(td[9].text)         # total points end of the season

            # put them into one dictionary
            if len(result) == 14:
                final_table.append(result)

    time.sleep(3)

    # put dictionary into a pandas dataframe with column names
    final_table = pd.DataFrame(final_table, columns=(['Team', 'P', 'W', 'D', 'L', 'GF', 'GA', 'HW', 'HGF', 'HGA', 'AW', 'AGF', 'AGA', 'PTS']))
    
    # Create features to generate poisson distribution
    final_table['H_Att'] = final_table['HGF'] / 19 / (final_table['HGF'].sum() / 380)
    final_table['A_Att'] = final_table['AGF'] / 19 / (final_table['AGF'].sum() / 380)
    final_table['H_Def'] = final_table['HGA'] / 19 / (final_table['HGA'].sum() / 380) #* table_13['DIS'] * table_13['CS'] / 38
    final_table['A_Def'] = final_table['AGA'] / 19 / (final_table['AGA'].sum() / 380) #* table_13['DIS'] * table_13['CS'] / 38
    
    # change index as a team names
    final_table = final_table.set_index('Team')
    # put all seasons into a list
    season_tables.append(final_table)

# make a list as a dictionary that has keys = season (2013, 2014, ..., 2017)
dict_seasons = {str(2012 + i) : season_tables[i] for i in range(0,6)}

In [10]:
dict_seasons['2012'] # EPL Season 2012/2013


,P,W,D,L,GF,GA,HW,HGF,HGA,AW,AGF,AGA,PTS,H_Att,A_Att,H_Def,A_Def
Team,,,,,,,,,,,,,,,,,
Manchester Utd,38,28,5,5,86,43,16,45,19,12,41,24,89,1.520270,1.740977,0.806794,0.810811
Manchester City,38,23,9,6,66,34,14,41,15,9,25,19,78,1.385135,1.061571,0.636943,0.641892
Chelsea,38,22,9,7,75,39,12,41,16,10,34,23,75,1.385135,1.443737,0.679406,0.777027
Arsenal,38,21,10,7,72,37,11,47,23,10,25,14,73,1.587838,1.061571,0.976645,0.472973
Tottenham,38,21,9,8,66,46,11,29,18,10,37,28,72,0.979730,1.571125,0.764331,0.945946
Everton,38,16,15,7,55,40,12,33,17,4,22,23,63,1.114865,0.934183,0.721868,0.777027
Liverpool,38,16,13,9,71,43,9,33,16,7,38,27,61,1.114865,1.613588,0.679406,0.912162
West Bromwich,38,14,7,17,53,57,9,32,25,5,21,32,49,1.081081,0.891720,1.061571,1.081081
Swansea City,38,11,13,14,47,51,6,28,26,5,19,25,46,0.945946,0.806794,1.104034,0.844595


### Step 3: Getting Cleans Sheets

In [11]:
cs_tables = []
seasons = ['2012-13', '2013-14', '2014-15', '2015-16', '2016-17', '2017-18']
for year in seasons:
    #load data from soccerstats.com by years (2013 will mean 13/14)
    table_url = 'https://www.sportsmole.co.uk/football/premier-league/' + year + '/best-defence.html'
    res = requests.get(table_url, headers={'User-agent': 'slee'})
   
    if res.status_code != 200:
        print('Status not 200', res.status_code)
        break
        
    soup = BeautifulSoup(res.content, 'lxml')

    # get a table
    table = soup.find('table', {'class':'leaguetable full'})
    tr = table.find_all('tr')
    cs_table = []

    for i in range(1,(len(tr))):
        result = {}

        td = tr[i].find_all('td')

        if td:
            result['Team'] = td[1].text.strip()          # goals allowed
            result['CS'] = int(td[4].text)         # total points end of the season

            # put them into one dictionary
            if len(result) == 2:
                cs_table.append(result)

    time.sleep(3)

    # put dictionary into a pandas dataframe with column names
    cs_table = pd.DataFrame(cs_table, columns=(['Team', 'CS']))
    # change index as a team names
    cs_table = cs_table.set_index('Team')
    cs_table.rename(index=team_name, inplace=True)
    # put all seasons into a list
    cs_tables.append(cs_table)

# make a list as a dictionary that has keys = season (2013, 2014, ..., 2017)
dict_cs = {str(2012 + i) : cs_tables[i] for i in range(0,6)}

In [12]:
dict_cs['2013']

,CS
Team,
Chelsea,18
Manchester City,16
Everton,15
Arsenal,17
Manchester Utd,13
Southampton,15
Crystal Palace,12
Liverpool,10
Tottenham,14


### Step 4: Getting 'Discipline' Points

In [13]:
dis_tables = []

for year in range(2012, 2018):
    # load data from transfermarkt.co.uk by years (2013 means 13/14)
    table_url = 'https://www.transfermarkt.co.uk/premier-league/fairnesstabelle/wettbewerb/GB1/saison_id/' + str(year) + '/plus/1'
    res = requests.get(table_url, headers={'User-agent': 'slsl'})

    if res.status_code != 200:
        print('Status not 200', res.status_code)

    soup = BeautifulSoup(res.content, 'lxml')

    # get a table
    table = soup.find('table', {'class': 'items'})
    tbody = table.find('tbody')
    tr = tbody.find_all('tr')

    dis_table = []

    for i in range(0,(len(tr))):
        result = {}

        td = tr[i].find_all('td')

        if td:
            result['Team'] = td[2].text.strip()     # team name
            result['P'] = int(td[3].text)           # games played
            result['YC'] = int(td[5].text)          # total number of yellow cards during the season
            result['RC'] = int(td[8].text)          # red cards
            # DIS = (0.5 * YC + 2 * RC) / P
            # higher 'DIS', the team is more likely to have cards during a match
            result['DIS'] = (int(td[5].text) * 0.5 + int(td[8].text) * 2) / int(td[3].text)    

            # put them into one dictionary
            if len(result) == 5:
                dis_table.append(result)

    time.sleep(3)

    # put dictionary into a pandas dataframe with column names
    dis_table = pd.DataFrame(dis_table, columns=(['Team', 'P', 'YC', 'RC', 'DIS']))
    # change index as a team names
    dis_table = dis_table.set_index('Team')
    dis_table.rename(index=team_name, inplace=True)
    # put all seasons into a list
    dis_tables.append(dis_table)
    
# make a list as a dictionary that has keys = season (2012, 2013, 2014, ..., 2017)
dict_dis = {str(2012 + i) : dis_tables[i] for i in range(0,6)}

In [14]:
dict_dis['2012']


,P,YC,RC,DIS
Team,,,,
Reading,38,45,1,0.644737
Southampton,38,43,2,0.671053
Chelsea,38,49,3,0.802632
Arsenal,38,40,5,0.789474
Manchester Utd,38,57,1,0.802632
Fulham,38,48,3,0.789474
Tottenham,38,54,2,0.815789
Swansea City,38,56,2,0.842105
Liverpool,38,54,2,0.815789


### Step 4: Merge Standing and Discipline

In [15]:
# merge dict_seasons, dict_dis
temp_pd = []
for year in range(2012, 2018):
    temp_pd.append(pd.merge(dict_seasons[str(year)], dict_dis[str(year)][['YC', 'RC', 'DIS']], left_index=True, right_index=True, how='outer'))
dict_past_seasons = {str(2012 + i) : temp_pd[i] for i in range(0,6)}

new_temp = []

for year in range(2012, 2018):
    new_temp.append(pd.merge(dict_past_seasons[str(year)], dict_cs[str(year)][['CS']], left_index=True, right_index=True, how='outer'))

dict_final_seasons = {str(2012 + i) : new_temp[i] for i in range(0,6)}



In [19]:
dict_final_seasons['2015'].T

Team,Arsenal,Aston Villa,Bournemouth,Chelsea,Crystal Palace,Everton,Leicester City,Liverpool,Manchester City,Manchester Utd,Newcastle Utd,Norwich City,Southampton,Stoke City,Sunderland,Swansea City,Tottenham,Watford,West Bromwich,West Ham Utd
P,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000
W,20.000000,3.000000,11.000000,12.000000,11.000000,11.000000,23.000000,16.000000,19.000000,19.000000,9.000000,9.000000,18.000000,14.000000,9.000000,12.000000,19.000000,12.000000,10.000000,16.000000
D,11.000000,8.000000,9.000000,14.000000,9.000000,14.000000,12.000000,12.000000,9.000000,9.000000,10.000000,7.000000,9.000000,9.000000,12.000000,11.000000,13.000000,9.000000,13.000000,14.000000
L,7.000000,27.000000,18.000000,12.000000,18.000000,13.000000,3.000000,10.000000,10.000000,10.000000,19.000000,22.000000,11.000000,15.000000,17.000000,15.000000,6.000000,17.000000,15.000000,8.000000
GF,65.000000,27.000000,45.000000,59.000000,39.000000,59.000000,68.000000,63.000000,71.000000,49.000000,44.000000,39.000000,59.000000,41.000000,48.000000,42.000000,69.000000,40.000000,34.000000,65.000000
GA,36.000000,76.000000,67.000000,53.000000,51.000000,55.000000,36.000000,50.000000,41.000000,35.000000,65.000000,67.000000,41.000000,55.000000,62.000000,52.000000,35.000000,50.000000,48.000000,51.000000
HW,12.000000,2.000000,5.000000,5.000000,6.000000,6.000000,12.000000,8.000000,12.000000,12.000000,7.000000,6.000000,11.000000,8.000000,6.000000,8.000000,10.000000,6.000000,6.000000,9.000000
HGF,31.000000,14.000000,23.000000,32.000000,19.000000,35.000000,35.000000,33.000000,47.000000,27.000000,32.000000,26.000000,39.000000,22.000000,23.000000,20.000000,35.000000,20.000000,20.000000,34.000000
HGA,11.000000,35.000000,34.000000,30.000000,23.000000,30.000000,18.000000,22.000000,21.000000,9.000000,24.000000,30.000000,22.000000,24.000000,20.000000,20.000000,15.000000,19.000000,26.000000,26.000000
AW,8.000000,1.000000,6.000000,7.000000,5.000000,5.000000,11.000000,8.000000,7.000000,7.000000,2.000000,3.000000,7.000000,6.000000,3.000000,4.000000,9.000000,6.000000,4.000000,7.000000


In [17]:
for season in range(2012, 2018):
    dict_final_seasons[str(season)].to_csv('./Data/epl_table_' + str(season-2000) + str(season-1999) + '.csv')

### Step 5: Create a Scoring Model (Poisson Distribution)

In [18]:
def score_percentage(dataframe, hometeam, awayteam):
    home_avg = dataframe['HGF'].sum()/380
    away_avg = dataframe['HGA'].sum()/380
    
    home_score = float(dataframe[dataframe.index == hometeam]['H_Att']) * float(dataframe[dataframe.index == awayteam]['A_Def']) * home_avg
    away_score = float(dataframe[dataframe.index == awayteam]['A_Att']) * float(dataframe[dataframe.index == hometeam]['H_Def']) * away_avg

    score = []
    # maximum score for a team is 5
    for goals in range(0, 6):
        scores = {}
        scores['Home'] = (poisson.pmf(goals, home_score) * 100) # Hometeam score
        scores['Away'] = (poisson.pmf(goals, away_score) * 100) # away score

        if len(scores) == 2:
                    score.append(scores)
    score = pd.DataFrame(score, columns=(['Home', 'Away']))
    home_w = 0
    away_w = 0
    draw = 0
    for home in range(1, len(score)):
        for away in range(0, home):
            home_w += (score['Home'][home] * score['Away'][away] / 10000)

    for away in range(1, len(score)):
        for home in range(0, away):
            away_w += (score['Home'][home] * score['Away'][away] / 10000)

    for home in range(0, len(score)):
        away = home
        draw += (score['Home'][home] * score['Away'][away] / 10000)
        
    return home_w, away_w, draw
